In [ ]:
#@title 🎧 Download Narration Audio & Play Introduction
import os as _os
if not _os.path.exists("/content/narration"):
    !pip install -q gdown
    import gdown
    gdown.download(id="1ventuhdj998YNr_9KusKPNX2VFJg7Av1", output="/content/narration.zip", quiet=False)
    !unzip -q /content/narration.zip -d /content/narration
    !rm /content/narration.zip
    print(f"Loaded {len(_os.listdir('/content/narration'))} narration segments")
else:
    print("Narration audio already loaded.")

from IPython.display import Audio, display
display(Audio("/content/narration/00_intro.mp3"))


In [ ]:
#@title 🎧 Listen: Setup
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_setup.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

# Data Parallelism & ZeRO: Splitting Data Across GPUs
## Notebook 2 of 6 — *5D Parallelism from Scratch*

**Vizuara AI — Google Colab Notebook**

In this notebook, we will implement Data Parallelism from scratch. We will simulate 4 "virtual GPUs" on a single device, split mini-batches across them, implement the AllReduce gradient averaging algorithm, and watch all four models converge identically. Then, we will build a ZeRO memory simulator to understand how the Zero Redundancy Optimizer eliminates the massive memory waste of vanilla DP.

**Estimated time: ~45 minutes**

**What you will learn:**
- Why Data Parallelism is the most fundamental distributed training strategy
- How AllReduce gradient averaging keeps models synchronized
- Why vanilla DP wastes memory and how ZeRO (Stages 1-3) fixes it
- The exact memory formulas used in production training systems

In [ ]:
# Setup — run this cell first
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import copy
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset

# Reproducibility
torch.manual_seed(42)
np.random.seed(42)

%matplotlib inline

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"PyTorch version: {torch.__version__}")
print(f"Device: {device}")
print("All libraries loaded successfully!")

In [ ]:
#@title 🎧 Listen: Why It Matters
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_why_it_matters.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 1. Why Does This Matter?

Data Parallelism is the bread and butter of distributed training. Every single large model trained today — GPT-4, Llama 3, DeepSeek-V3, Gemini — uses Data Parallelism as one of its core strategies. It is the simplest parallelism technique, and understanding it is the foundation for everything else in this series.

Here is the key idea: **if one GPU is too slow, use N GPUs — each processing a different chunk of data, then share what they learned.**

By the end of this notebook, you will see something beautiful: four separate models, each training on different data, yet converging to *exactly the same weights* — as if a single model had trained on all the data at once.

Let us start with a teaser. We will run the full Data Parallel training simulation and see all four models converge. If this does not make sense yet, do not worry — we will build every component from scratch.

In [ ]:
# Quick teaser: What we will build by the end of this notebook
# (just run this cell to see the final result — we will build it step by step)

print("=" * 60)
print("  DATA PARALLELISM: 4 Virtual GPUs Training Together")
print("=" * 60)
print()
print("By the end of this notebook, you will see:")
print("  1. Four model copies, each seeing different data")
print("  2. AllReduce averaging their gradients every step")
print("  3. All four loss curves perfectly overlapping")
print("  4. ZeRO memory savings from 112 GB to 28 GB per GPU")
print()
print("Let us build it!")

In [ ]:
#@title 🎧 Listen: Restaurant Analogy
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_restaurant_analogy.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 2. Building Intuition

Before we write any equations or code, let us develop a deep intuition for what Data Parallelism actually does.

### The Restaurant Analogy

Imagine you run a restaurant that has just received 1,000 orders. You have one chef. Each order takes 1 minute. That is 1,000 minutes — over 16 hours.

What is the simplest fix? **Hire 4 chefs.** Give each chef a **complete copy of the recipe book** and split the orders: Chef 0 gets orders 1–250, Chef 1 gets orders 251–500, and so on.

Each chef cooks independently. But here is the critical part: at the end of each round, all 4 chefs **share notes**. "I found that adding a pinch more salt works better." "I discovered the oven should be 10 degrees hotter." They average their learnings, and everyone updates their recipe book identically.

This is Data Parallelism:
- **4 chefs** = 4 GPUs
- **Complete recipe book** = full copy of the model
- **Split orders** = split the mini-batch
- **Share notes** = AllReduce (gradient averaging)
- **Update recipe identically** = synchronized weight update

### The Key Insight

Notice something important: each chef still needs the **full recipe book**. Hiring more chefs does not make the recipe book smaller. It only makes cooking faster.

This is both the strength and the weakness of Data Parallelism:
- **Strength**: Linearly scales throughput (4 GPUs → ~4x faster training)
- **Weakness**: Does not reduce per-GPU memory at all

### Think About This

If a 7B parameter model needs 112 GB of memory for training, and you use Data Parallelism across 4 GPUs, how much memory does each GPU need?

The answer: still **112 GB each**. Data Parallelism does not help with memory — it only helps with speed. Each GPU holds a full copy of everything.

What if the recipe book itself is too big for one chef's desk? That is the memory problem DP does not solve. What if, instead of each chef having a full recipe book, we *tore it into sections*? That is ZeRO — and we will get there.

In [ ]:
#@title 🎧 Listen: Math Allreduce
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_math_allreduce.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 3. The Mathematics

Now let us formalize these intuitions with precise equations. We will derive everything step by step.

### 3.1 The AllReduce Equation

Each GPU $i$ processes its own mini-batch $B_i$ and computes a gradient $g_i$. To keep all GPUs synchronized, we average these gradients:

$$g_{\text{avg}} = \frac{1}{N} \sum_{i=1}^{N} g_i$$

Here, $N$ is the number of GPUs (or "workers"), and $g_i$ is the gradient computed by GPU $i$.

**Why does averaging work?** Because the expected value of the average gradient equals the gradient of the full batch:

$$\mathbb{E}[g_{\text{avg}}] = \mathbb{E}\left[\frac{1}{N} \sum_{i=1}^{N} g_i\right] = \frac{1}{N} \sum_{i=1}^{N} \mathbb{E}[g_i] = \mathbb{E}[g_{\text{full batch}}]$$

Each $g_i$ is an unbiased estimate of the true gradient. Averaging $N$ unbiased estimates gives another unbiased estimate — but with $N$ times more data, so it is a better estimate (lower variance).

### 3.2 A Concrete Numerical Example

Let us plug in some simple numbers. Suppose we have 4 GPUs, and for a single parameter, each GPU computes a gradient:

- GPU 0: $g_0 = 0.5$
- GPU 1: $g_1 = 0.3$
- GPU 2: $g_2 = 0.7$
- GPU 3: $g_3 = 0.1$

The AllReduce operation computes:

$$g_{\text{avg}} = \frac{0.5 + 0.3 + 0.7 + 0.1}{4} = \frac{1.6}{4} = 0.4$$

All four GPUs now use $g_{\text{avg}} = 0.4$ to update their weights. Since they started with identical weights and apply the identical update, they remain perfectly synchronized.

In [ ]:
# Let us verify the AllReduce math computationally
gradients = [0.5, 0.3, 0.7, 0.1]
N = len(gradients)
g_avg = sum(gradients) / N

print("AllReduce — Numerical Example")
print("-" * 40)
for i, g in enumerate(gradients):
    print(f"  GPU {i}: gradient = {g}")
print(f"\n  g_avg = ({' + '.join(str(g) for g in gradients)}) / {N}")
print(f"        = {sum(gradients)} / {N}")
print(f"        = {g_avg}")
print(f"\nAll 4 GPUs now use g_avg = {g_avg}")

In [ ]:
#@title 🎧 Listen: Weight Update
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_weight_update.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 3.3 The Weight Update

After AllReduce, every GPU applies the same update rule:

$$w_{t+1} = w_t - \eta \cdot g_{\text{avg}}$$

where $\eta$ is the learning rate. Since all GPUs start with the same $w_t$ and apply the same $g_{\text{avg}}$, they all arrive at the same $w_{t+1}$.

This is the invariant that Data Parallelism must maintain: **all model copies have identical weights at every step.**

In [ ]:
# Demonstrate the weight update invariant
lr = 0.01
w_initial = 1.0  # Same starting weight on all GPUs

print("Weight Update After AllReduce")
print("-" * 40)
print(f"  Initial weight (all GPUs): w = {w_initial}")
print(f"  Learning rate: eta = {lr}")
print(f"  Averaged gradient: g_avg = {g_avg}")
print(f"\n  w_new = w - eta x g_avg")
print(f"        = {w_initial} - {lr} x {g_avg}")
print(f"        = {w_initial - lr * g_avg}")
print(f"\nAll 4 GPUs arrive at w = {w_initial - lr * g_avg}")
print("They remain perfectly synchronized!")

In [ ]:
#@title 🎧 Listen: Memory Problem
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/06_memory_problem.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 3.4 The Memory Problem

Now let us understand why vanilla Data Parallelism does not save memory. With mixed-precision training using the Adam optimizer, each parameter requires:

| Component | Precision | Bytes per param |
|-----------|-----------|-----------------|
| Model weights | fp16 | 2 |
| Gradients | fp16 | 2 |
| Optimizer: fp32 weight copy | fp32 | 4 |
| Optimizer: first moment (m) | fp32 | 4 |
| Optimizer: second moment (v) | fp32 | 4 |
| **Total** | | **16** |

For a model with $P$ parameters:

$$M_{\text{per GPU}} = 16P \text{ bytes}$$

With vanilla DP, every GPU stores all of this. For a 7B model:

$$M = 7 \times 10^9 \times 16 = 112 \text{ GB per GPU}$$

Four GPUs? Still 112 GB each. A thousand GPUs? Still 112 GB each. Vanilla DP does not help with memory at all.

In [ ]:
# Memory calculation for vanilla Data Parallelism
def vanilla_dp_memory_gb(num_params_billions, num_gpus):
    """Calculate per-GPU memory for vanilla DP (no savings)."""
    P = num_params_billions * 1e9
    bytes_per_param = 16  # 2 (weights) + 2 (grads) + 12 (optimizer)
    total_bytes = P * bytes_per_param
    per_gpu_gb = total_bytes / (1024**3)
    return per_gpu_gb

# 7B model with different GPU counts
for n_gpus in [1, 4, 8, 64]:
    mem = vanilla_dp_memory_gb(7, n_gpus)
    print(f"  7B model, {n_gpus:>2} GPUs -> {mem:.1f} GB per GPU (no savings!)")

print("\nAdding more GPUs does NOT reduce memory with vanilla DP.")
print("Each GPU still holds the full model, gradients, and optimizer.")

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/07_build_model.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 4. Let Us Build It — Component by Component

Now we will implement Data Parallelism from scratch. We will use a small 2-layer MLP trained on MNIST to keep things fast, but the principles are exactly the same as training a 70B model on 1,000 GPUs.

### 4.1 The Model

Our model is intentionally simple — two linear layers with ReLU activation. The goal is not the model architecture but the parallelism mechanics.

In [ ]:
class SimpleMLP(nn.Module):
    """A 2-layer MLP for MNIST classification.

    Small enough to run 4 copies on any hardware,
    but complex enough to demonstrate real DP behavior.
    """
    def __init__(self, input_dim=784, hidden_dim=256, output_dim=10):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten: (B, 1, 28, 28) -> (B, 784)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Create and inspect the model
model = SimpleMLP().to(device)
total_params = sum(p.numel() for p in model.parameters())
print(f"Model: SimpleMLP")
print(f"  Parameters: {total_params:,}")
print(f"  fc1: {784} x {256} + {256} bias = {784*256 + 256:,}")
print(f"  fc2: {256} x {10} + {10} bias = {256*10 + 10:,}")
print(f"\nModel created on {device}")

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/08_load_mnist.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.2 Loading MNIST

We will use a subset of MNIST to keep training fast. The key thing to understand is that in real DP, different GPUs would receive different shards of the dataset. Here, we will simulate this by explicitly splitting each mini-batch.

In [ ]:
# Load MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST(
    root='./data', train=True, download=True, transform=transform
)

# Use a subset for speed (8000 samples is plenty for our demo)
train_subset = Subset(train_dataset, range(8000))

# Global batch size = 128, which we will split across 4 "GPUs"
GLOBAL_BATCH_SIZE = 128
train_loader = DataLoader(
    train_subset, batch_size=GLOBAL_BATCH_SIZE, shuffle=True
)

print(f"Training samples: {len(train_subset):,}")
print(f"Global batch size: {GLOBAL_BATCH_SIZE}")
print(f"Batches per epoch: {len(train_loader)}")
print(f"\nMNIST loaded successfully")

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/09_virtual_gpus.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.3 Creating Virtual GPUs

This is where the simulation begins. In real Data Parallelism, each GPU has its own model copy. We simulate this by creating 4 independent copies of our MLP. The crucial detail: **all copies start with identical weights.**

In [ ]:
NUM_GPUS = 4

def create_virtual_gpus(base_model, num_gpus):
    """Create N identical copies of a model to simulate N GPUs.

    In real DP, each copy lives on a separate GPU.
    Here, they all live on the same device but are independent nn.Modules.
    """
    models = []
    for i in range(num_gpus):
        model_copy = copy.deepcopy(base_model)
        models.append(model_copy)
    return models

# Create 4 "virtual GPUs" — each with an identical model copy
base_model = SimpleMLP().to(device)
virtual_gpus = create_virtual_gpus(base_model, NUM_GPUS)

We also need a helper to verify that all model copies remain synchronized throughout training. This function checks that every parameter is identical across all copies.

In [ ]:
def check_weight_sync(models):
    """Check that all model copies have identical weights."""
    reference = list(models[0].parameters())
    for i in range(1, len(models)):
        for p_ref, p_i in zip(reference, list(models[i].parameters())):
            if not torch.allclose(p_ref, p_i):
                return False
    return True

print(f"Created {NUM_GPUS} virtual GPUs")
print(f"Weights synchronized: {check_weight_sync(virtual_gpus)}")
print(f"All {NUM_GPUS} models start with identical weights")

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/10_split_batch.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.4 Splitting the Batch

In real DP, a data loader distributes different mini-batches to different GPUs. We simulate this by taking a single batch of 128 samples and splitting it into 4 micro-batches of 32.

In [ ]:
def split_batch(data, targets, num_gpus):
    """Split a batch evenly across virtual GPUs.

    Args:
        data: (B, C, H, W) tensor — the full mini-batch
        targets: (B,) tensor — the labels
        num_gpus: number of GPUs to split across

    Returns:
        List of (micro_data, micro_targets) tuples, one per GPU
    """
    micro_batch_size = len(data) // num_gpus
    splits = []
    for i in range(num_gpus):
        start = i * micro_batch_size
        end = start + micro_batch_size
        splits.append((data[start:end], targets[start:end]))
    return splits

Let us demonstrate the split on a sample batch.

In [ ]:
# Demonstrate the split
sample_data, sample_targets = next(iter(train_loader))
splits = split_batch(sample_data, sample_targets, NUM_GPUS)

print(f"Global batch: {sample_data.shape[0]} samples")
print(f"Split across {NUM_GPUS} GPUs:")
for i, (d, t) in enumerate(splits):
    print(f"  GPU {i}: {d.shape[0]} samples (indices {i*32}-{(i+1)*32-1})")
print(f"\nEach GPU gets {splits[0][0].shape[0]} samples")

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/11_forward_backward.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.5 Forward and Backward on Each GPU

Each virtual GPU independently runs forward and backward passes on its own micro-batch. This is the "embarrassingly parallel" part — no communication needed here.

In [ ]:
def forward_backward_on_all_gpus(models, splits, criterion):
    """Run forward + backward on each GPU's micro-batch.

    Each model processes its own data independently.
    After this, each model has its own gradients stored
    in .grad attributes.

    Returns:
        List of losses (one per GPU)
    """
    losses = []
    for i, (model, (data, targets)) in enumerate(
        zip(models, splits)
    ):
        model.zero_grad()
        data, targets = data.to(device), targets.to(device)
        output = model(data)
        loss = criterion(output, targets)
        loss.backward()
        losses.append(loss.item())
    return losses

Let us run the forward and backward passes on all 4 virtual GPUs and inspect the resulting losses.

In [ ]:
# Run forward + backward on all 4 GPUs
criterion = nn.CrossEntropyLoss()
losses = forward_backward_on_all_gpus(virtual_gpus, splits, criterion)

print("Forward + Backward Pass (Independent)")
print("-" * 40)
for i, loss in enumerate(losses):
    print(f"  GPU {i}: loss = {loss:.4f}")

Now let us peek at the gradients before AllReduce. Each GPU has different gradients because it saw different data.

In [ ]:
# Show that gradients differ across GPUs BEFORE AllReduce
print("Gradients BEFORE AllReduce (first 5 values of fc1.weight.grad)")
print("-" * 60)
for i, model in enumerate(virtual_gpus):
    grads = model.fc1.weight.grad.flatten()[:5]
    grad_str = ", ".join(f"{g:.4f}" for g in grads)
    print(f"  GPU {i}: [{grad_str}]")

print("\nGradients are DIFFERENT — each GPU saw different data!")

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/12_allreduce_impl.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.6 Implementing AllReduce

This is the heart of Data Parallelism. AllReduce takes the gradients from all GPUs, averages them, and writes the averaged gradient back to every GPU. After this operation, all models have identical gradients and will take identical weight update steps.

In [ ]:
def allreduce_gradients(models):
    """AllReduce: Average gradients across all model copies.

    This is the key operation in Data Parallelism.
    After this, all models have identical gradients.

    In real distributed training, this is implemented using
    efficient ring-allreduce algorithms over NCCL.
    Here we simulate it directly.
    """
    num_models = len(models)

    # For each parameter, average the gradients across all models
    for param_group in zip(*[model.parameters() for model in models]):
        # Stack all gradients: shape (num_models, *param_shape)
        all_grads = torch.stack([p.grad for p in param_group])
        # Compute the mean across models (dimension 0)
        avg_grad = all_grads.mean(dim=0)
        # Write the averaged gradient back to ALL models
        for p in param_group:
            p.grad.copy_(avg_grad)

# Apply AllReduce
allreduce_gradients(virtual_gpus)
print("AllReduce complete — gradients averaged across all GPUs")

**Visualization Checkpoint 1:** Let us see the effect of AllReduce. We will verify that all GPUs now have identical gradients.

In [ ]:
# Show that gradients are NOW identical across all GPUs
print("Gradients AFTER AllReduce (first 5 values of fc1.weight.grad)")
print("-" * 60)
for i, model in enumerate(virtual_gpus):
    grads = model.fc1.weight.grad.flatten()[:5]
    grad_str = ", ".join(f"{g:.4f}" for g in grads)
    print(f"  GPU {i}: [{grad_str}]")

# Verify they are truly identical
all_identical = True
ref_grads = list(virtual_gpus[0].parameters())
for model in virtual_gpus[1:]:
    for p_ref, p in zip(ref_grads, model.parameters()):
        if not torch.allclose(p_ref.grad, p.grad):
            all_identical = False
            break

print(f"\nAll gradients identical: {all_identical}")
print("Now all GPUs will take the exact same weight update step!")

In [ ]:
#@title 🎧 What to Look For
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/13_allreduce_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Now let us visualize the before and after of AllReduce with a side-by-side plot. We re-create fresh models and re-run forward/backward to capture both states.

In [ ]:
# Re-create models and re-run forward/backward
demo_models = create_virtual_gpus(base_model, NUM_GPUS)
demo_losses = forward_backward_on_all_gpus(demo_models, splits, criterion)

# Collect "before" gradients (first 20 values of fc1.weight.grad)
num_vals = 20
before_grads = []
for m in demo_models:
    before_grads.append(
        m.fc1.weight.grad.flatten()[:num_vals].detach().cpu().numpy()
    )

# Apply AllReduce and collect "after" gradients
allreduce_gradients(demo_models)
after_grads = []
for m in demo_models:
    after_grads.append(
        m.fc1.weight.grad.flatten()[:num_vals].detach().cpu().numpy()
    )

Now we plot the before vs after comparison.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
x = np.arange(num_vals)
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

# Plot BEFORE
for i in range(NUM_GPUS):
    axes[0].plot(x, before_grads[i], 'o-', color=colors[i],
                 label=f'GPU {i}', markersize=4, alpha=0.8)
axes[0].set_title('BEFORE AllReduce', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Parameter index')
axes[0].set_ylabel('Gradient value')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot AFTER
for i in range(NUM_GPUS):
    axes[1].plot(x, after_grads[i], 'o-', color=colors[i],
                 label=f'GPU {i}', markersize=4, alpha=0.8)
axes[1].set_title('AFTER AllReduce', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Parameter index')
axes[1].set_ylabel('Gradient value')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

Add a title and display the figure.

In [ ]:
# 📊 Visualization
plt.suptitle('AllReduce: Gradient Averaging Across GPUs',
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("Left: each GPU has different gradients (different data)")
print("Right: after AllReduce, all GPUs have the SAME averaged gradient")

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/14_training_loop.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.7 The Full Data Parallel Training Loop

Now we put it all together. The training loop follows this pattern every step:

1. Split the mini-batch across GPUs
2. Each GPU runs forward + backward independently
3. AllReduce averages the gradients
4. Each GPU applies the same weight update
5. All models remain perfectly synchronized

In [ ]:
def train_data_parallel(num_gpus, num_epochs=3, lr=0.01):
    """Complete Data Parallel training simulation.

    Simulates N GPUs by maintaining N model copies on one device.
    Returns per-GPU loss histories for visualization.
    """
    # Create identical model copies
    base = SimpleMLP().to(device)
    models = create_virtual_gpus(base, num_gpus)

    # Each GPU has its own optimizer (operating on its local model)
    optimizers = [optim.SGD(m.parameters(), lr=lr) for m in models]
    criterion = nn.CrossEntropyLoss()

    # Track per-GPU loss curves
    loss_histories = [[] for _ in range(num_gpus)]
    step = 0

    for epoch in range(num_epochs):
        for batch_idx, (data, targets) in enumerate(train_loader):
            micro_batches = split_batch(data, targets, num_gpus)
            for i, (model, (mb_data, mb_targets)) in enumerate(
                zip(models, micro_batches)
            ):
                optimizers[i].zero_grad()
                mb_data = mb_data.to(device)
                mb_targets = mb_targets.to(device)
                loss = criterion(model(mb_data), mb_targets)
                loss.backward()
                loss_histories[i].append(loss.item())

The second half of the training loop performs AllReduce and the weight update. We also check synchronization at the end of each epoch.

In [ ]:
            # Step 3: AllReduce — average gradients
            allreduce_gradients(models)

            # Step 4: Each GPU applies the same update
            for opt in optimizers:
                opt.step()

            step += 1

        # End-of-epoch check
        synced = check_weight_sync(models)
        avg_loss = np.mean([h[-1] for h in loss_histories])
        print(f"  Epoch {epoch+1}/{num_epochs} — "
              f"Avg loss: {avg_loss:.4f} — "
              f"Synced: {synced}")

    return models, loss_histories

Let us now run the complete training simulation.

In [ ]:
print("Training with simulated 4-way Data Parallelism...")
print("=" * 50)
trained_models, loss_histories = train_data_parallel(
    num_gpus=NUM_GPUS, num_epochs=3, lr=0.01
)
print("=" * 50)
print("Training complete!")

In [ ]:
#@title 🎧 What to Look For
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/15_loss_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


**Visualization Checkpoint 2:** The moment of truth. All four loss curves should overlap perfectly after AllReduce.

In [ ]:
# Per-GPU loss curves (raw and smoothed)
fig, axes = plt.subplots(1, 2, figsize=(16, 5))
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

# Left: Raw per-GPU loss curves
for i in range(NUM_GPUS):
    axes[0].plot(loss_histories[i], color=colors[i],
                 label=f'GPU {i}', alpha=0.7, linewidth=1)
axes[0].set_title('Per-GPU Loss Curves', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Training step')
axes[0].set_ylabel('Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

The smoothed view makes the convergence pattern clearer.

In [ ]:
# 📊 Right: Overlay showing convergence (smoothed)
fig2, ax_smooth = plt.subplots(figsize=(8, 5))
window = 10
for i in range(NUM_GPUS):
    smoothed = np.convolve(loss_histories[i],
                           np.ones(window)/window, mode='valid')
    ax_smooth.plot(smoothed, color=colors[i],
                   label=f'GPU {i}', alpha=0.8, linewidth=2)
ax_smooth.set_title('Smoothed Loss (window=10)', fontsize=14, fontweight='bold')
ax_smooth.set_xlabel('Training step')
ax_smooth.set_ylabel('Loss')
ax_smooth.legend()
ax_smooth.grid(True, alpha=0.3)

plt.suptitle('Data Parallelism: All GPUs Converge Together',
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("The loss curves differ slightly because each GPU sees different data,")
print("but they converge together because AllReduce synchronizes them.")

In [ ]:
#@title 🎧 Listen: Weight Sync Verify
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/16_weight_sync_verify.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Let us also verify that the models have truly identical weights at the end of training.

In [ ]:
# Verify final weight synchronization
print("Final Weight Synchronization Check")
print("-" * 50)

max_diff = 0.0
for name, p_ref in trained_models[0].named_parameters():
    for i in range(1, NUM_GPUS):
        p_i = dict(trained_models[i].named_parameters())[name]
        diff = (p_ref - p_i).abs().max().item()
        max_diff = max(max_diff, diff)

print(f"  Maximum weight difference across all GPUs: {max_diff:.2e}")

if max_diff < 1e-6:
    print(f"\nAll {NUM_GPUS} models have IDENTICAL weights!")
    print("Data Parallelism maintains perfect synchronization.")
else:
    print(f"\nSmall numerical differences detected (floating point).")
    print(f"This is normal — the key point is they converge together.")

In [ ]:
#@title 🎧 Before You Start
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/17_todo1_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 5. Your Turn — TODO Exercises

Now it is your turn to implement the core components. These exercises will cement your understanding of Data Parallelism.

### TODO 1: Implement the AllReduce Averaging Step

Below is a function signature with a docstring. Your task: **fill in the body** to average gradients across all model copies. This is the most important function in Data Parallelism.

In [ ]:
def my_allreduce(models):
    """Implement AllReduce gradient averaging.

    For each parameter across all models:
      1. Collect the gradient from each model
      2. Compute the mean
      3. Write the mean gradient back to ALL models

    Hint: Loop over zip(*[model.parameters() for model in models])
          to iterate over corresponding parameters simultaneously.
    """
    # ============ TODO ============
    # Step 1: Stack all gradients into a tensor using torch.stack
    # Step 2: Compute the mean across dim=0
    # Step 3: Copy the averaged gradient to ALL models using .copy_()
    # ==============================

    num_models = len(models)

    for param_group in zip(*[m.parameters() for m in models]):
        all_grads = ???  # YOUR CODE HERE
        avg_grad = ???   # YOUR CODE HERE
        for p in param_group:
            pass          # YOUR CODE HERE — use p.grad.copy_(...)

In [ ]:
#@title 🎧 Listen: Todo1 Verify
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/18_todo1_verify.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Run the verification cell below to check your implementation. Do not modify this cell.

In [ ]:
# Verification for TODO 1
print("Verifying your AllReduce implementation...")
print("-" * 50)

# Create fresh models and compute gradients
test_base = SimpleMLP().to(device)
test_models = create_virtual_gpus(test_base, 4)
test_data, test_targets = next(iter(train_loader))
test_splits = split_batch(test_data, test_targets, 4)
_ = forward_backward_on_all_gpus(
    test_models, test_splits, nn.CrossEntropyLoss()
)

Now we compare your implementation against the reference.

In [ ]:
# Save expected result using reference implementation
ref_models = create_virtual_gpus(test_base, 4)
_ = forward_backward_on_all_gpus(
    ref_models, test_splits, nn.CrossEntropyLoss()
)
allreduce_gradients(ref_models)

# Apply student's implementation
my_allreduce(test_models)

# Check per-parameter correctness
all_correct = True
for (name, p_ref), (_, p_stu) in zip(
    ref_models[0].named_parameters(),
    test_models[0].named_parameters()
):
    if p_stu.grad is None:
        print(f"  {name}: gradient is None — did you forget to set it?")
        all_correct = False
    elif not torch.allclose(p_ref.grad, p_stu.grad, atol=1e-6):
        print(f"  {name}: gradient mismatch!")
        all_correct = False

Check that all GPUs received the same averaged gradient.

In [ ]:
# Check cross-GPU gradient consistency
if all_correct:
    for i in range(1, 4):
        for p0, pi in zip(test_models[0].parameters(),
                          test_models[i].parameters()):
            if not torch.allclose(p0.grad, pi.grad, atol=1e-6):
                print(f"  GPU 0 and GPU {i} have different gradients!")
                all_correct = False
                break

if all_correct:
    print("  AllReduce implementation is CORRECT!")
    print("  All GPUs have identical averaged gradients!")
else:
    print("\n  Hint: Stack gradients -> mean(dim=0) -> copy back to all")

In [ ]:
#@title 🎧 Before You Start
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/19_todo2_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### TODO 2: Implement ZeRO Stage 1 Optimizer State Partitioning

In ZeRO Stage 1, each GPU only stores 1/N of the optimizer states. After AllReduce of gradients, each GPU updates only its shard of parameters using its local optimizer, then all GPUs broadcast (AllGather) the updated weights so everyone has the full model.

Your task: implement the `zero_stage1_step` function that performs one training step with ZeRO-1 style optimizer sharding.

In [ ]:
def zero_stage1_step(models, optimizers, param_shards, num_gpus):
    """Perform one ZeRO Stage 1 optimizer step.

    In ZeRO-1, each GPU owns a shard of the optimizer states.
    After AllReduce of gradients:
      1. Each GPU updates ONLY its assigned parameter shard
      2. AllGather broadcasts updated weights to all GPUs
    """
    # ============ TODO ============
    # Step 1: AllReduce gradients (already implemented — call it)
    # Step 2: For each GPU, update ONLY its assigned param shard
    # Step 3: AllGather — copy updated params from owner to all
    # ==============================

    # Step 1: AllReduce gradients across all models
    allreduce_gradients(models)

    # Step 2: Each GPU updates only its own shard
    for gpu_id in range(num_gpus):
        my_indices = param_shards[gpu_id]
        opt = optimizers[gpu_id]
        opt.zero_grad()
        for idx in my_indices:
            src_param = list(models[gpu_id].parameters())[idx]
            src_param.grad = ???  # YOUR CODE HERE

    result = ???  # YOUR CODE HERE — perform the optimizer step

The second part of `zero_stage1_step`: AllGather broadcasts the updated weights from each owner GPU to all others.

In [ ]:
    # Step 3: AllGather — broadcast updated weights
    for gpu_id in range(num_gpus):
        my_indices = param_shards[gpu_id]
        for idx in my_indices:
            updated = list(models[gpu_id].parameters())[idx]
            for other_gpu in range(num_gpus):
                if other_gpu != gpu_id:
                    target = list(models[other_gpu].parameters())[idx]
                    target.data.copy_(???)  # YOUR CODE HERE

    return models

In [ ]:
#@title 🎧 Listen: Todo2 Verify
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/20_todo2_verify.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Run the verification cell below to check your ZeRO Stage 1 implementation.

In [ ]:
# Verification for TODO 2
print("Verifying your ZeRO Stage 1 implementation...")
print("-" * 50)

# Setup: create models, assign shards
z1_base = SimpleMLP().to(device)
z1_models = create_virtual_gpus(z1_base, NUM_GPUS)

# Count parameters and assign shards
all_param_list = list(z1_models[0].parameters())
num_total_params = len(all_param_list)
print(f"  Total parameter groups: {num_total_params}")

Now we create the shard assignments and run the step.

In [ ]:
# Assign parameter indices to GPUs round-robin
param_shards = {g: [] for g in range(NUM_GPUS)}
for idx in range(num_total_params):
    owner = idx % NUM_GPUS
    param_shards[owner].append(idx)

for g in range(NUM_GPUS):
    print(f"  GPU {g} owns param indices: {param_shards[g]}")

# Create per-GPU optimizers
z1_optimizers = [
    optim.SGD(m.parameters(), lr=0.01) for m in z1_models
]

# Run one forward-backward pass
z1_data, z1_targets = next(iter(train_loader))
z1_splits = split_batch(z1_data, z1_targets, NUM_GPUS)
_ = forward_backward_on_all_gpus(
    z1_models, z1_splits, nn.CrossEntropyLoss()
)

Now apply your ZeRO-1 step and check if models stay synchronized.

In [ ]:
# Apply student's ZeRO-1 step
try:
    zero_stage1_step(z1_models, z1_optimizers, param_shards, NUM_GPUS)
    synced = check_weight_sync(z1_models)
    assert synced, "Models are not synchronized after ZeRO-1 step!"
    print("\n  ZeRO Stage 1 step is CORRECT!")
    print("  All GPUs synchronized after shard-based update + AllGather!")
except Exception as e:
    print(f"\n  Error: {e}")
    print("  Hint: Each GPU updates its shard, then broadcasts to others.")

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/21_unified_demo.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 6. Putting It All Together

Now let us combine everything — Data Parallelism with AllReduce and ZeRO-style memory accounting — into a unified demonstration. We will train with simulated 4-way DP, track memory as if each stage of ZeRO were active, and compare the results.

### 6.1 Unified DP + ZeRO Training Demo

This function wraps our training loop with a ZeRO memory tracker, so you can see both the training convergence and the memory implications side by side.

In [ ]:
def zero_memory_per_gpu_bytes(num_params, num_gpus, stage):
    """Compute per-GPU memory in bytes for a given ZeRO stage.

    Args:
        num_params: total model parameters (P)
        num_gpus: number of GPUs (N)
        stage: 0 (vanilla DP), 1, 2, or 3
    Returns:
        Memory in bytes (float)
    """
    P, N = num_params, num_gpus
    if stage == 0:
        return 16 * P
    elif stage == 1:
        return 4 * P + 12 * P / N
    elif stage == 2:
        return 2 * P + (2 * P + 12 * P) / N
    elif stage == 3:
        return 16 * P / N
    else:
        raise ValueError(f"Invalid stage: {stage}")

Now we define the unified training function that logs both convergence and ZeRO memory statistics.

In [ ]:
def train_dp_with_zero_tracking(num_gpus, num_epochs=3, lr=0.01):
    """Train with DP and report ZeRO memory stats at each epoch."""
    base = SimpleMLP().to(device)
    models = create_virtual_gpus(base, num_gpus)
    optimizers = [optim.SGD(m.parameters(), lr=lr) for m in models]
    criterion = nn.CrossEntropyLoss()

    total_params = sum(p.numel() for p in models[0].parameters())
    loss_histories = [[] for _ in range(num_gpus)]
    step = 0

    print(f"Model parameters: {total_params:,}")
    print(f"Simulating {num_gpus}-way Data Parallelism")
    print("-" * 55)

    for epoch in range(num_epochs):
        for data, targets in train_loader:
            micro_batches = split_batch(data, targets, num_gpus)
            for i, (model, (d, t)) in enumerate(
                zip(models, micro_batches)
            ):
                optimizers[i].zero_grad()
                d, t = d.to(device), t.to(device)
                loss = criterion(model(d), t)
                loss.backward()
                loss_histories[i].append(loss.item())
            allreduce_gradients(models)
            for opt in optimizers:
                opt.step()
            step += 1

At the end of each epoch, we print the ZeRO memory comparison.

In [ ]:
        synced = check_weight_sync(models)
        avg_loss = np.mean([h[-1] for h in loss_histories])
        print(f"\n  Epoch {epoch+1}/{num_epochs} | "
              f"Loss: {avg_loss:.4f} | Synced: {synced}")

        # ZeRO memory report (scaled to 7B for realism)
        P_7B = 7e9
        print(f"  Memory per GPU (if 7B model, {num_gpus} GPUs):")
        for stg, name in [(0, "Vanilla DP"), (1, "ZeRO-1"),
                          (2, "ZeRO-2"), (3, "ZeRO-3/FSDP")]:
            mem_gb = zero_memory_per_gpu_bytes(P_7B, num_gpus, stg) / (1024**3)
            bar = "#" * int(mem_gb / 3)
            print(f"    {name:<14s}: {mem_gb:6.1f} GB  {bar}")

    return models, loss_histories, total_params

Let us run the unified demo.

In [ ]:
print("=" * 55)
print("  UNIFIED DEMO: Data Parallelism + ZeRO Memory Tracking")
print("=" * 55)

final_models, final_histories, n_params = train_dp_with_zero_tracking(
    num_gpus=NUM_GPUS, num_epochs=3, lr=0.01
)
print("\n" + "=" * 55)
print("Demo complete!")

In [ ]:
#@title 🎧 What to Look For
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/22_unified_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 6.2 Final Comparison Visualization

We combine the training convergence and ZeRO memory savings into a single summary figure.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

# Left: Smoothed loss curves from unified training
window = 10
for i in range(NUM_GPUS):
    smoothed = np.convolve(
        final_histories[i], np.ones(window)/window, mode='valid'
    )
    ax1.plot(smoothed, color=colors[i],
             label=f'GPU {i}', linewidth=1.5, alpha=0.8)
ax1.set_title('DP Training: All GPUs Converge', fontsize=13, fontweight='bold')
ax1.set_xlabel('Step')
ax1.set_ylabel('Loss')
ax1.legend(fontsize=9)
ax1.grid(True, alpha=0.3)

Now the right panel: ZeRO memory comparison.

In [ ]:
# Right: ZeRO memory breakdown for 7B model
P_7B = 7e9
stage_names = ['Vanilla DP', 'ZeRO-1', 'ZeRO-2', 'ZeRO-3']
stage_ids = [0, 1, 2, 3]
mem_vals = [
    zero_memory_per_gpu_bytes(P_7B, NUM_GPUS, s) / (1024**3)
    for s in stage_ids
]
bar_colors = ['#95a5a6', '#3498db', '#2ecc71', '#e74c3c']

ax2.bar(range(4), mem_vals, color=bar_colors, edgecolor='white')
ax2.axhline(y=80, color='red', linestyle='--', alpha=0.5, linewidth=1.5)
ax2.text(3.4, 81, 'A100 80GB', color='red', fontsize=9)
ax2.set_xticks(range(4))
ax2.set_xticklabels(stage_names, fontsize=9)
ax2.set_title('ZeRO Memory (7B, 4 GPUs)', fontsize=13, fontweight='bold')
ax2.set_ylabel('Per-GPU Memory (GB)')
for i, v in enumerate(mem_vals):
    ax2.text(i, v + 1.5, f'{v:.0f}', ha='center', fontsize=10,
             fontweight='bold')
ax2.grid(True, axis='y', alpha=0.3)

Display the combined figure and print key takeaways.

In [ ]:
# 📊 Visualization
plt.suptitle('Data Parallelism + ZeRO: Training & Memory',
             fontsize=15, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

savings = (1 - mem_vals[3] / mem_vals[0]) * 100
print(f"DP trains correctly: all {NUM_GPUS} GPUs converge together.")
print(f"ZeRO-3 saves {savings:.0f}% memory vs vanilla DP.")
print(f"This is how production systems train models that do not fit on one GPU.")

In [ ]:
#@title 🎧 Listen: Zero Deep Dive
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/23_zero_deep_dive.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 7. ZeRO: Eliminating Redundancy

Now let us go deeper into ZeRO — one of the most impactful ideas in distributed training. The core insight is devastatingly simple: **in vanilla DP, every GPU stores the exact same optimizer states, gradients, and weights. That is N-fold redundancy, and it is completely unnecessary.**

### 7.1 Understanding Each ZeRO Stage

Let us walk through what each GPU stores at each stage.

**Vanilla DP (no ZeRO):** Each of 4 GPUs stores:
- Full weights (2P bytes)
- Full gradients (2P bytes)
- Full optimizer states (12P bytes)
- Total: 16P bytes per GPU, 64P bytes across the cluster

The optimizer states alone are 12 bytes per parameter (fp32 weight copy + Adam first moment + Adam second moment). For 4 GPUs, that is 4 identical copies of 12P bytes = 48P bytes wasted on redundancy.

**ZeRO Stage 1 — Shard Optimizer States:**
Each GPU stores only 1/4 of the optimizer states. After the AllReduce of gradients, each GPU updates only its shard of the optimizer and then broadcasts (AllGather) the updated weights.

**ZeRO Stage 2 — Shard Optimizer + Gradients:**
On top of Stage 1, gradients are also sharded. Each GPU only keeps the gradients for the parameters whose optimizer states it owns.

**ZeRO Stage 3 — Shard Everything (FSDP):**
Even the model weights are sharded. Each GPU only stores 1/N of the weights. When a layer needs weights it does not own, it fetches them via AllGather, uses them for the forward/backward pass, and then discards them.

In [ ]:
# ZeRO memory formulas — reference implementation
def zero_memory_reference(num_params_billions, num_gpus):
    """Compute per-GPU memory for all ZeRO stages.

    Returns a dict of {stage_name: memory_in_GB}.
    """
    P = num_params_billions * 1e9
    N = num_gpus

    results = {}
    results['Vanilla DP'] = 16 * P
    results['ZeRO-1'] = 4 * P + 12 * P / N
    results['ZeRO-2'] = 2 * P + (2 * P + 12 * P) / N
    results['ZeRO-3 (FSDP)'] = 16 * P / N

    # Convert bytes to GB
    return {k: v / (1024**3) for k, v in results.items()}

Let us calculate and display the memory for a 7B model with 4 GPUs.

In [ ]:
# Calculate for 7B model with 4 GPUs
mem_4gpu = zero_memory_reference(7, 4)

print("ZeRO Memory Breakdown — 7B Model, 4 GPUs")
print("=" * 50)
for stage, mem in mem_4gpu.items():
    bar = "#" * int(mem / 2)
    print(f"  {stage:<20s}: {mem:6.1f} GB  {bar}")

savings = (1 - mem_4gpu['ZeRO-3 (FSDP)'] / mem_4gpu['Vanilla DP']) * 100
print(f"\nZeRO-3 reduces memory by {savings:.0f}% compared to vanilla DP!")
print(f"From {mem_4gpu['Vanilla DP']:.1f} GB -> "
      f"{mem_4gpu['ZeRO-3 (FSDP)']:.1f} GB per GPU")

In [ ]:
#@title 🎧 What to Look For
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/24_zero_stacked_bar.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


**Visualization Checkpoint 3:** A stacked bar chart showing exactly what each GPU stores under each ZeRO stage.

In [ ]:
# ZeRO Memory Comparison — stacked bar chart setup
fig, ax = plt.subplots(figsize=(12, 7))

P = 7e9  # 7B parameters
N = 4    # 4 GPUs

stages = ['Vanilla DP', 'ZeRO-1', 'ZeRO-2', 'ZeRO-3\n(FSDP)']

weights_gb = [
    2 * P / 1024**3,
    2 * P / 1024**3,
    2 * P / 1024**3,
    2 * P / N / 1024**3,
]
gradients_gb = [
    2 * P / 1024**3,
    2 * P / 1024**3,
    2 * P / N / 1024**3,
    2 * P / N / 1024**3,
]
optimizer_gb = [
    12 * P / 1024**3,
    12 * P / N / 1024**3,
    12 * P / N / 1024**3,
    12 * P / N / 1024**3,
]

Now we draw the stacked bar chart.

In [ ]:
x = np.arange(len(stages))
width = 0.5

bars1 = ax.bar(x, weights_gb, width, label='Weights (fp16)',
               color='#3498db', edgecolor='white', linewidth=0.5)
bars2 = ax.bar(x, gradients_gb, width, bottom=weights_gb,
               label='Gradients (fp16)', color='#2ecc71',
               edgecolor='white', linewidth=0.5)
bars3 = ax.bar(x, optimizer_gb, width,
               bottom=[w+g for w, g in zip(weights_gb, gradients_gb)],
               label='Optimizer States (fp32)', color='#e74c3c',
               edgecolor='white', linewidth=0.5)

# Add total labels on top of each bar
for i in range(len(stages)):
    total = weights_gb[i] + gradients_gb[i] + optimizer_gb[i]
    ax.text(i, total + 1.5, f'{total:.1f} GB',
            ha='center', fontsize=12, fontweight='bold')

# A100 memory line
ax.axhline(y=80, color='red', linestyle='--', linewidth=2, alpha=0.7)
ax.text(3.3, 82, 'A100 80GB', color='red', fontsize=11, fontweight='bold')

Format axes and display the chart.

In [ ]:
# 📊 Visualization
ax.set_xlabel('Strategy', fontsize=13)
ax.set_ylabel('Per-GPU Memory (GB)', fontsize=13)
ax.set_title('ZeRO: Progressive Memory Savings\n'
             '7B Model with 4 GPUs (Mixed Precision + Adam)',
             fontsize=15, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(stages, fontsize=12)
ax.legend(loc='upper right', fontsize=11)
ax.set_ylim(0, 120)
ax.grid(True, axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("Key insight: Vanilla DP exceeds the A100's 80 GB limit!")
print("ZeRO-1 barely fits. ZeRO-2 and ZeRO-3 fit comfortably.")

In [ ]:
#@title 🎧 What to Look For
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/25_zero_scaling.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 7.2 Scaling ZeRO: More GPUs, More Savings

The beauty of ZeRO is that memory savings scale with the number of GPUs. Let us see how per-GPU memory changes as we add more GPUs.

In [ ]:
# ZeRO memory scaling with number of GPUs
gpu_counts = [1, 2, 4, 8, 16, 32, 64]
P_billions = 7  # 7B model

fig, ax = plt.subplots(figsize=(12, 6))

stage_colors = {
    'Vanilla DP': '#95a5a6',
    'ZeRO-1': '#3498db',
    'ZeRO-2': '#2ecc71',
    'ZeRO-3 (FSDP)': '#e74c3c',
}

for stage_name in ['Vanilla DP', 'ZeRO-1', 'ZeRO-2', 'ZeRO-3 (FSDP)']:
    memories = []
    for n in gpu_counts:
        mem = zero_memory_reference(P_billions, n)
        memories.append(mem[stage_name])
    ax.plot(gpu_counts, memories, 'o-', color=stage_colors[stage_name],
            label=stage_name, linewidth=2, markersize=8)

Add reference lines and labels to the scaling plot.

In [ ]:
# 📊 A100 reference line
ax.axhline(y=80, color='red', linestyle='--', alpha=0.5, linewidth=1.5)
ax.text(50, 82, 'A100 80GB', color='red', fontsize=10)

ax.set_xlabel('Number of GPUs', fontsize=13)
ax.set_ylabel('Per-GPU Memory (GB)', fontsize=13)
ax.set_title('ZeRO Memory Scaling — 7B Model',
             fontsize=15, fontweight='bold')
ax.legend(fontsize=11)
ax.set_xscale('log', base=2)
ax.set_xticks(gpu_counts)
ax.set_xticklabels(gpu_counts)
ax.grid(True, alpha=0.3)
ax.set_ylim(0, 120)

plt.tight_layout()
plt.show()

print("Vanilla DP (gray) stays flat — no memory savings ever!")
print("ZeRO-3 (red) drops to ~1.6 GB with 64 GPUs.")
print("This is how we train 70B+ models: shard everything.")

In [ ]:
#@title 🎧 Listen: Communication Tradeoff
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/26_communication_tradeoff.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 7.3 The Communication Trade-off

ZeRO saves memory, but there is a cost: **more communication**. Let us understand this trade-off.

- **Vanilla DP**: One AllReduce per step (gradients only). Communication volume: $2P$ bytes.
- **ZeRO-1**: Same communication as vanilla DP. The optimizer sharding is "free" — each GPU only updates its own shard, then broadcasts via AllGather. Roughly $2P$ bytes.
- **ZeRO-2**: Same as ZeRO-1 — gradient sharding uses ReduceScatter instead of AllReduce, but the total volume is similar. Roughly $2P$ bytes.
- **ZeRO-3**: Adds AllGather for weights in both forward and backward passes. Communication roughly $3P$ bytes — a 1.5x increase, but memory drops to $16P/N$.

In [ ]:
# Communication volume comparison — setup data
P_billions = 7
P = P_billions * 1e9

comm_data = {
    'Vanilla DP': 2 * P * 2,
    'ZeRO-1':     2 * P * 2,
    'ZeRO-2':     2 * P * 2,
    'ZeRO-3':     3 * P * 2,
}
comm_gb = {k: v / 1024**3 for k, v in comm_data.items()}

N = 4
mem_data = zero_memory_reference(P_billions, N)

Now let us plot the memory vs communication trade-off side by side.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Memory savings (with N=4)
stages_ordered = ['Vanilla DP', 'ZeRO-1', 'ZeRO-2', 'ZeRO-3 (FSDP)']
mem_vals = [mem_data[s] for s in stages_ordered]
colors_ordered = ['#95a5a6', '#3498db', '#2ecc71', '#e74c3c']

ax1.bar(range(4), mem_vals, color=colors_ordered, edgecolor='white')
ax1.set_xticks(range(4))
ax1.set_xticklabels(
    ['Vanilla\nDP', 'ZeRO-1', 'ZeRO-2', 'ZeRO-3\n(FSDP)'], fontsize=10
)
ax1.set_ylabel('Per-GPU Memory (GB)', fontsize=12)
ax1.set_title('Memory (lower = better)', fontsize=13, fontweight='bold')
ax1.grid(True, axis='y', alpha=0.3)
for i, v in enumerate(mem_vals):
    ax1.text(i, v + 1, f'{v:.0f}', ha='center', fontsize=11,
             fontweight='bold')

Now the right panel: communication volume comparison.

In [ ]:
# 📊 Communication volume
comm_stages = ['Vanilla\nDP', 'ZeRO-1', 'ZeRO-2', 'ZeRO-3']
comm_vals = [comm_gb['Vanilla DP'], comm_gb['ZeRO-1'],
             comm_gb['ZeRO-2'], comm_gb['ZeRO-3']]
ax2.bar(range(4), comm_vals, color=colors_ordered, edgecolor='white')
ax2.set_xticks(range(4))
ax2.set_xticklabels(comm_stages, fontsize=10)
ax2.set_ylabel('Communication per Step (GB)', fontsize=12)
ax2.set_title('Communication (lower = better)',
              fontsize=13, fontweight='bold')
ax2.grid(True, axis='y', alpha=0.3)
for i, v in enumerate(comm_vals):
    ax2.text(i, v + 0.3, f'{v:.0f}', ha='center', fontsize=11,
             fontweight='bold')

plt.suptitle('ZeRO Trade-off: Memory vs Communication\n'
             '7B Model, 4 GPUs', fontsize=15, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("ZeRO-3 saves 75% memory but increases communication by ~50%.")
print("In practice, this trade-off is almost always worth it.")

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/27_sharding_sim.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 7.4 Simulating ZeRO Sharding

Let us simulate what ZeRO actually does under the hood. We will take a small parameter tensor and show how it gets sharded across GPUs at each stage.

In [ ]:
def simulate_zero_sharding(param_name, param_size, num_gpus):
    """Simulate ZeRO sharding for a single parameter.

    Shows what each GPU stores at each ZeRO stage.
    """
    shard_size = param_size // num_gpus

    print(f"\nParameter: {param_name} ({param_size} elements)")
    print("=" * 60)

    # Vanilla DP
    print("\n  Vanilla DP:")
    for g in range(num_gpus):
        print(f"    GPU {g}: weights[0:{param_size}] + "
              f"grads[0:{param_size}] + opt[0:{param_size}]")

    # ZeRO Stage 1: shard optimizer
    print("\n  ZeRO Stage 1 (shard optimizer):")
    for g in range(num_gpus):
        os, oe = g * shard_size, (g+1) * shard_size
        print(f"    GPU {g}: weights[0:{param_size}] + "
              f"grads[0:{param_size}] + opt[{os}:{oe}]")

Continuing the simulation for Stages 2 and 3.

In [ ]:
    # ZeRO Stage 2: shard optimizer + gradients
    print("\n  ZeRO Stage 2 (shard optimizer + gradients):")
    for g in range(num_gpus):
        s, e = g * shard_size, (g+1) * shard_size
        print(f"    GPU {g}: weights[0:{param_size}] + "
              f"grads[{s}:{e}] + opt[{s}:{e}]")

    # ZeRO Stage 3: shard everything
    print("\n  ZeRO Stage 3 / FSDP (shard everything):")
    for g in range(num_gpus):
        s, e = g * shard_size, (g+1) * shard_size
        print(f"    GPU {g}: weights[{s}:{e}] + "
              f"grads[{s}:{e}] + opt[{s}:{e}]")

simulate_zero_sharding("fc1.weight", param_size=1024, num_gpus=4)

In [ ]:
#@title 🎧 What to Look For
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/28_sharding_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Now let us build a visual diagram of the sharding. We draw each GPU's stored components at each ZeRO stage.

In [ ]:
# Visual simulation of ZeRO sharding — setup
fig, axes = plt.subplots(4, 1, figsize=(14, 10))

stages = ['Vanilla DP\n(Full Replication)',
          'ZeRO Stage 1\n(Shard Optimizer)',
          'ZeRO Stage 2\n(Shard Opt + Grad)',
          'ZeRO Stage 3 / FSDP\n(Shard Everything)']

N = 4
components = [
    ('Weights\n(fp16)', 2, '#3498db'),
    ('Gradients\n(fp16)', 2, '#2ecc71'),
    ('Optimizer\n(fp32)', 12, '#e74c3c'),
]

Draw the sharding diagram for all 4 stages.

In [ ]:
for ax_idx, (ax, stage_name) in enumerate(zip(axes, stages)):
    for gpu_id in range(N):
        y = gpu_id * 1.5
        x_start = 0
        for comp_idx, (comp_name, comp_width, comp_color) in enumerate(
            components
        ):
            if ax_idx == 0:
                ew = comp_width
            elif ax_idx == 1:
                ew = comp_width / N if comp_idx == 2 else comp_width
            elif ax_idx == 2:
                ew = comp_width / N if comp_idx >= 1 else comp_width
            else:
                ew = comp_width / N

            rect = plt.Rectangle(
                (x_start, y), ew, 1.0, facecolor=comp_color,
                edgecolor='white', linewidth=1.5, alpha=0.85
            )
            ax.add_patch(rect)

            if ew < comp_width:
                ghost = plt.Rectangle(
                    (x_start, y), comp_width, 1.0,
                    facecolor='none', edgecolor=comp_color,
                    linewidth=1, linestyle='--', alpha=0.3
                )
                ax.add_patch(ghost)
            x_start += comp_width

Add labels and formatting to each subplot.

In [ ]:
        ax.text(-0.5, y + 0.5, f'GPU {gpu_id}', ha='right',
                va='center', fontsize=9, fontweight='bold')

    # Total memory per GPU
    if ax_idx == 0:
        total = 16
    elif ax_idx == 1:
        total = 4 + 12/N
    elif ax_idx == 2:
        total = 2 + 14/N
    else:
        total = 16/N

    ax.text(17, 2.25, f'{total:.1f}x\nper GPU', ha='center', va='center',
            fontsize=11, fontweight='bold',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

    ax.set_xlim(-3, 19)
    ax.set_ylim(-0.3, 6.3)
    ax.set_title(stage_name, fontsize=11, fontweight='bold', loc='left')
    ax.set_xticks([])
    ax.set_yticks([])
    for spine in ax.spines.values():
        spine.set_visible(False)

Finally, add a legend and display the complete figure.

In [ ]:
# 📊 Add legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='#3498db', edgecolor='white',
          label='Weights (2 bytes/param)'),
    Patch(facecolor='#2ecc71', edgecolor='white',
          label='Gradients (2 bytes/param)'),
    Patch(facecolor='#e74c3c', edgecolor='white',
          label='Optimizer (12 bytes/param)'),
]
fig.legend(handles=legend_elements, loc='lower center', ncol=3,
           fontsize=10, bbox_to_anchor=(0.5, -0.02))

plt.suptitle('ZeRO: What Each GPU Stores at Each Stage',
             fontsize=15, fontweight='bold')
plt.tight_layout()
plt.subplots_adjust(bottom=0.06)
plt.show()

print("Solid bars = what this GPU actually stores")
print("Dashed outlines = what it would store in vanilla DP")
print("ZeRO-3 reduces each GPU's storage to just 1/N of the total!")

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/29_final_training.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 8. Final Output

Let us put everything together into one complete, clean Data Parallel training run. We will:
1. Train a model with simulated 4-way Data Parallelism on MNIST
2. Show all 4 GPU loss curves converging identically
3. Evaluate final accuracy
4. Show the ZeRO memory comparison chart

In [ ]:
print("=" * 60)
print("  FINAL: Complete Data Parallel Training Simulation")
print("=" * 60)
print()

# ---- Training Configuration ----
NUM_GPUS_FINAL = 4
NUM_EPOCHS_FINAL = 3
LR_FINAL = 0.01
GLOBAL_BS = 128

print(f"Configuration:")
print(f"  Virtual GPUs:     {NUM_GPUS_FINAL}")
print(f"  Epochs:           {NUM_EPOCHS_FINAL}")
print(f"  Global batch:     {GLOBAL_BS}")
print(f"  Micro-batch:      {GLOBAL_BS // NUM_GPUS_FINAL} per GPU")
print(f"  Learning rate:    {LR_FINAL}")
print(f"  Model:            SimpleMLP (784 -> 256 -> 10)")
print()

In [ ]:
# Full training run with detailed logging
base_model_final = SimpleMLP().to(device)
models_final = create_virtual_gpus(base_model_final, NUM_GPUS_FINAL)
optimizers_final = [optim.SGD(m.parameters(), lr=LR_FINAL)
                    for m in models_final]
criterion_final = nn.CrossEntropyLoss()

# Track metrics
all_loss_histories = [[] for _ in range(NUM_GPUS_FINAL)]
sync_checks = []
step_count = 0

print("Training...")
print("-" * 50)
for epoch in range(NUM_EPOCHS_FINAL):
    epoch_losses = [[] for _ in range(NUM_GPUS_FINAL)]

    for data, targets in train_loader:
        micro_batches = split_batch(data, targets, NUM_GPUS_FINAL)
        for i, (model, (d, t)) in enumerate(
            zip(models_final, micro_batches)
        ):
            optimizers_final[i].zero_grad()
            d, t = d.to(device), t.to(device)
            out = model(d)
            loss = criterion_final(out, t)
            loss.backward()
            all_loss_histories[i].append(loss.item())
            epoch_losses[i].append(loss.item())

Continuing the training loop: AllReduce, weight update, and epoch logging.

In [ ]:
        # AllReduce
        allreduce_gradients(models_final)

        # Update
        for opt in optimizers_final:
            opt.step()

        step_count += 1

    # Check synchronization
    synced = check_weight_sync(models_final)
    sync_checks.append(synced)
    avg_epoch_loss = np.mean([np.mean(el) for el in epoch_losses])
    print(f"  Epoch {epoch+1}/{NUM_EPOCHS_FINAL} — "
          f"Loss: {avg_epoch_loss:.4f} — Synced: {synced}")

print(f"\nTotal steps: {step_count}")
print(f"All epochs synced: {all(sync_checks)}")

In [ ]:
#@title 🎧 Listen: Final Eval
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/30_final_eval.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Now we evaluate the final accuracy on the test set. We use GPU 0's model since all models are identical.

In [ ]:
# Evaluate accuracy on test set using GPU 0's model
test_dataset = datasets.MNIST(
    root='./data', train=False, download=True, transform=transform
)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

models_final[0].eval()
correct = 0
total = 0

with torch.no_grad():
    for data, targets in test_loader:
        data, targets = data.to(device), targets.to(device)
        output = models_final[0](data)
        _, predicted = output.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

accuracy = 100.0 * correct / total
print(f"\nTest Accuracy: {accuracy:.2f}%")
print(f"(Using GPU 0's model — all GPUs have identical weights)")

In [ ]:
#@title 🎧 What to Look For
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/31_final_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Finally, let us create the complete summary visualization with four panels.

In [ ]:
# Final Visualization — panel 1: loss curves
fig = plt.figure(figsize=(16, 10))
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

ax1 = fig.add_subplot(2, 2, 1)
window = 10
for i in range(NUM_GPUS_FINAL):
    smoothed = np.convolve(all_loss_histories[i],
                           np.ones(window)/window, mode='valid')
    ax1.plot(smoothed, color=colors[i], label=f'GPU {i}',
             linewidth=1.5, alpha=0.8)
ax1.set_title('Loss Curves (All 4 GPUs)', fontsize=13, fontweight='bold')
ax1.set_xlabel('Step')
ax1.set_ylabel('Loss')
ax1.legend(fontsize=9)
ax1.grid(True, alpha=0.3)

Panel 2: weight differences across GPUs.

In [ ]:
# Panel 2: Weight difference
ax2 = fig.add_subplot(2, 2, 2)
weight_diffs = []
for i in range(1, NUM_GPUS_FINAL):
    diff = 0
    for p0, pi in zip(models_final[0].parameters(),
                      models_final[i].parameters()):
        diff += (p0 - pi).abs().max().item()
    weight_diffs.append(diff)
ax2.bar([f'GPU 0 vs {i}' for i in range(1, NUM_GPUS_FINAL)],
        weight_diffs, color=colors[1:], edgecolor='white')
ax2.set_title('Final Weight Differences', fontsize=13, fontweight='bold')
ax2.set_ylabel('Max Absolute Difference')
ax2.ticklabel_format(style='scientific', axis='y', scilimits=(0,0))
ax2.grid(True, axis='y', alpha=0.3)

Panels 3 and 4: ZeRO memory and text summary.

In [ ]:
# Panel 3: ZeRO memory comparison
ax3 = fig.add_subplot(2, 2, 3)
mem_7b = zero_memory_reference(7, 4)
mem_values = list(mem_7b.values())
bar_colors = ['#95a5a6', '#3498db', '#2ecc71', '#e74c3c']
ax3.bar(range(4), mem_values, color=bar_colors, edgecolor='white')
ax3.axhline(y=80, color='red', linestyle='--', alpha=0.5, linewidth=1.5)
ax3.text(3.4, 81, 'A100', color='red', fontsize=9)
ax3.set_xticks(range(4))
ax3.set_xticklabels(['Vanilla\nDP', 'ZeRO-1', 'ZeRO-2', 'ZeRO-3'],
                     fontsize=9)
ax3.set_title('ZeRO Memory (7B, 4 GPUs)', fontsize=13, fontweight='bold')
ax3.set_ylabel('Per-GPU Memory (GB)')
for i, v in enumerate(mem_values):
    ax3.text(i, v + 1.5, f'{v:.0f}', ha='center', fontsize=10,
             fontweight='bold')
ax3.grid(True, axis='y', alpha=0.3)

Panel 4: text summary of all results.

In [ ]:
# 📊 Visualization
ax4 = fig.add_subplot(2, 2, 4)
ax4.axis('off')
sav = (1 - mem_7b['ZeRO-3 (FSDP)'] / mem_7b['Vanilla DP']) * 100
summary_text = (
    f"Data Parallelism Summary\n"
    f"{'=' * 35}\n\n"
    f"Virtual GPUs:        {NUM_GPUS_FINAL}\n"
    f"Training epochs:     {NUM_EPOCHS_FINAL}\n"
    f"Total steps:         {step_count}\n"
    f"Final test accuracy: {accuracy:.2f}%\n"
    f"Models synchronized: {'Yes' if all(sync_checks) else 'No'}\n\n"
    f"ZeRO-3 memory savings:\n"
    f"  Vanilla DP: {mem_7b['Vanilla DP']:.0f} GB/GPU\n"
    f"  ZeRO-3:     {mem_7b['ZeRO-3 (FSDP)']:.0f} GB/GPU\n"
    f"  Savings:    {sav:.0f}%"
)
ax4.text(0.1, 0.95, summary_text, transform=ax4.transAxes,
         fontsize=12, verticalalignment='top', fontfamily='monospace',
         bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.9))

plt.suptitle('Data Parallelism & ZeRO — Complete Results',
             fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Listen: Summary Print
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/32_summary_print.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


A final summary of everything we built in this notebook.

In [ ]:
print("Congratulations! You have implemented Data Parallelism from scratch!")
print()
print("Here is what you built:")
print("  1. Simulated 4 virtual GPUs with independent model copies")
print("  2. Split mini-batches across GPUs (128 -> 4x32)")
print("  3. Ran independent forward/backward passes")
print("  4. Implemented AllReduce gradient averaging")
print("  5. Verified all models converge identically")
print("  6. Computed ZeRO memory savings (Stages 1-3)")
print("  7. Combined DP + ZeRO in a unified demo")
print()
print(f"Final accuracy: {accuracy:.2f}%")
print()
print("Key takeaways:")
print("  - DP scales throughput linearly — 4 GPUs ~ 4x faster")
print("  - AllReduce ensures all models stay perfectly synchronized")
print("  - Vanilla DP wastes memory — every GPU stores everything")
print("  - ZeRO-3 (FSDP) shards everything -> 75% memory savings")
print()
print("Next up: Tensor Parallelism — splitting the model itself!")

In [ ]:
#@title 🎧 Listen: Reflection
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/33_reflection.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 9. Reflection and Next Steps

### Reflection Questions

Before moving on, think carefully about these questions. They test your understanding of the core concepts.

1. **Why does DP require AllReduce after every step?** What would happen if we skipped AllReduce for 10 steps and then did it once? Would the models diverge? Would they produce different predictions? Would they eventually reconverge?

2. **ZeRO-3 has the lowest memory but highest communication.** When is this trade-off worth it? Consider: What if the GPUs are connected by slow Ethernet instead of fast NVLink? What if the model is 7B vs 175B?

3. **If you have 1024 GPUs, does DP alone give you 1024x speedup?** Think about: the micro-batch size per GPU (global batch 1024 → 1 sample per GPU?), the communication overhead of AllReduce at scale, and the effect of very large batch sizes on convergence.

4. **Why is the gradient average mathematically equivalent to the full-batch gradient?** Think about: the linearity of expectation, the definition of the loss function as an average over samples, and when this equivalence might break down.

In [ ]:
#@title 🎧 Listen: Challenges
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/34_challenges.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### Optional Challenges

If you want to go further, try these extensions:

**Challenge 1: Gradient Compression**

Implement top-k gradient compression: only communicate the top 10% largest gradient values. This reduces communication volume by 90% at the cost of some convergence speed.

In [ ]:
# Starter code for gradient compression
def topk_compress(gradient, k_fraction=0.1):
    """Keep only the top-k% largest gradient values."""
    flat = gradient.flatten()
    k = max(1, int(len(flat) * k_fraction))
    # YOUR CODE: Find top-k values, zero out the rest
    # Hint: use torch.topk
    pass

**Challenge 2: Gradient Accumulation**

Implement gradient accumulation: accumulate gradients over 4 micro-batches before doing AllReduce. This simulates a larger effective batch size while using less memory per step.

**Challenge 3: Communication Volume Tracker**

Add instrumentation to the training loop that measures the exact number of bytes communicated in AllReduce at each step. Compare this to the theoretical minimum.

In [ ]:
#@title 🎧 Wrap-Up
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/35_closing.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


---

*This notebook is Part 2 of the Vizuara series on 5D Parallelism from Scratch.*

*Next: Notebook 3 — Tensor Parallelism: Splitting the Model Itself*

In [ ]:
#@title 💬 AI Teaching Assistant — Click ▶ to start
#@markdown This AI chatbot reads your notebook and can answer questions about any concept, code, or exercise.

import json as _json
import requests as _requests
from google.colab import output as _output
from IPython.display import display, HTML as _HTML, Markdown as _Markdown

# --- Read notebook content for context ---
def _get_notebook_context():
    try:
        from google.colab import _message
        nb = _message.blocking_request("get_ipynb", request="", timeout_sec=10)
        cells = nb.get("ipynb", {}).get("cells", [])
        parts = []
        for cell in cells:
            src = "".join(cell.get("source", []))
            tags = cell.get("metadata", {}).get("tags", [])
            if "chatbot" in tags:
                continue
            if src.strip():
                ct = cell.get("cell_type", "unknown")
                parts.append(f"[{ct.upper()}]\n{src}")
        return "\n\n---\n\n".join(parts)
    except Exception:
        return "Notebook content unavailable."

_NOTEBOOK_CONTEXT = _get_notebook_context()
_CHAT_HISTORY = []
_API_URL = "https://course-creator-brown.vercel.app/api/chat"

def _notebook_chat(question):
    global _CHAT_HISTORY
    try:
        resp = _requests.post(_API_URL, json={
            'question': question,
            'context': _NOTEBOOK_CONTEXT[:100000],
            'history': _CHAT_HISTORY[-10:],
        }, timeout=60)
        data = resp.json()
        answer = data.get('answer', 'Sorry, I could not generate a response.')
        _CHAT_HISTORY.append({'role': 'user', 'content': question})
        _CHAT_HISTORY.append({'role': 'assistant', 'content': answer})
        return answer
    except Exception as e:
        return f'Error connecting to teaching assistant: {str(e)}'

_output.register_callback('notebook_chat', _notebook_chat)

def ask(question):
    """Ask the AI teaching assistant a question about this notebook."""
    answer = _notebook_chat(question)
    display(_Markdown(answer))

print("\u2705 AI Teaching Assistant is ready!")
print("\U0001f4a1 Use the chat below, or call ask(\'your question\') in any cell.")

# --- Display chat widget ---
display(_HTML('''<style>
  .vc-wrap{font-family:-apple-system,BlinkMacSystemFont,'Segoe UI',Roboto,sans-serif;max-width:100%;border-radius:16px;overflow:hidden;box-shadow:0 4px 24px rgba(0,0,0,.12);background:#fff;border:1px solid #e5e7eb}
  .vc-hdr{background:linear-gradient(135deg,#667eea 0%,#764ba2 100%);color:#fff;padding:16px 20px;display:flex;align-items:center;gap:12px}
  .vc-avatar{width:42px;height:42px;background:rgba(255,255,255,.2);border-radius:50%;display:flex;align-items:center;justify-content:center;font-size:22px}
  .vc-hdr h3{font-size:16px;font-weight:600;margin:0}
  .vc-hdr p{font-size:12px;opacity:.85;margin:2px 0 0}
  .vc-msgs{height:420px;overflow-y:auto;padding:16px;background:#f8f9fb;display:flex;flex-direction:column;gap:10px}
  .vc-msg{display:flex;flex-direction:column;animation:vc-fade .25s ease}
  .vc-msg.user{align-items:flex-end}
  .vc-msg.bot{align-items:flex-start}
  .vc-bbl{max-width:85%;padding:10px 14px;border-radius:16px;font-size:14px;line-height:1.55;word-wrap:break-word}
  .vc-msg.user .vc-bbl{background:linear-gradient(135deg,#667eea 0%,#764ba2 100%);color:#fff;border-bottom-right-radius:4px}
  .vc-msg.bot .vc-bbl{background:#fff;color:#1a1a2e;border:1px solid #e8e8e8;border-bottom-left-radius:4px}
  .vc-bbl code{background:rgba(0,0,0,.07);padding:2px 6px;border-radius:4px;font-size:13px;font-family:'Fira Code',monospace}
  .vc-bbl pre{background:#1e1e2e;color:#cdd6f4;padding:12px;border-radius:8px;overflow-x:auto;margin:8px 0;font-size:13px}
  .vc-bbl pre code{background:none;padding:0;color:inherit}
  .vc-bbl h3,.vc-bbl h4{margin:10px 0 4px;font-size:15px}
  .vc-bbl ul,.vc-bbl ol{margin:4px 0;padding-left:20px}
  .vc-bbl li{margin:2px 0}
  .vc-chips{display:flex;flex-wrap:wrap;gap:8px;padding:0 16px 12px;background:#f8f9fb}
  .vc-chip{background:#fff;border:1px solid #d1d5db;border-radius:20px;padding:6px 14px;font-size:12px;cursor:pointer;transition:all .15s;color:#4b5563}
  .vc-chip:hover{border-color:#667eea;color:#667eea;background:#f0f0ff}
  .vc-input{display:flex;padding:12px 16px;background:#fff;border-top:1px solid #eee;gap:8px}
  .vc-input input{flex:1;padding:10px 16px;border:2px solid #e8e8e8;border-radius:24px;font-size:14px;outline:none;transition:border-color .2s}
  .vc-input input:focus{border-color:#667eea}
  .vc-input button{background:linear-gradient(135deg,#667eea 0%,#764ba2 100%);color:#fff;border:none;border-radius:50%;width:42px;height:42px;cursor:pointer;display:flex;align-items:center;justify-content:center;font-size:18px;transition:transform .1s}
  .vc-input button:hover{transform:scale(1.05)}
  .vc-input button:disabled{opacity:.5;cursor:not-allowed;transform:none}
  .vc-typing{display:flex;gap:5px;padding:4px 0}
  .vc-typing span{width:8px;height:8px;background:#667eea;border-radius:50%;animation:vc-bounce 1.4s infinite ease-in-out}
  .vc-typing span:nth-child(2){animation-delay:.2s}
  .vc-typing span:nth-child(3){animation-delay:.4s}
  @keyframes vc-bounce{0%,80%,100%{transform:scale(0)}40%{transform:scale(1)}}
  @keyframes vc-fade{from{opacity:0;transform:translateY(8px)}to{opacity:1;transform:translateY(0)}}
  .vc-note{text-align:center;font-size:11px;color:#9ca3af;padding:8px 16px 12px;background:#fff}
</style>
<div class="vc-wrap">
  <div class="vc-hdr">
    <div class="vc-avatar">&#129302;</div>
    <div>
      <h3>Vizuara Teaching Assistant</h3>
      <p>Ask me anything about this notebook</p>
    </div>
  </div>
  <div class="vc-msgs" id="vcMsgs">
    <div class="vc-msg bot">
      <div class="vc-bbl">&#128075; Hi! I've read through this entire notebook. Ask me about any concept, code block, or exercise &mdash; I'm here to help you learn!</div>
    </div>
  </div>
  <div class="vc-chips" id="vcChips">
    <span class="vc-chip" onclick="vcAsk(this.textContent)">Explain the main concept</span>
    <span class="vc-chip" onclick="vcAsk(this.textContent)">Help with the TODO exercise</span>
    <span class="vc-chip" onclick="vcAsk(this.textContent)">Summarize what I learned</span>
  </div>
  <div class="vc-input">
    <input type="text" id="vcIn" placeholder="Ask about concepts, code, exercises..." />
    <button id="vcSend" onclick="vcSendMsg()">&#10148;</button>
  </div>
  <div class="vc-note">AI-generated &middot; Verify important information &middot; <a href="#" onclick="vcClear();return false" style="color:#667eea">Clear chat</a></div>
</div>
<script>
(function(){
  var msgs=document.getElementById('vcMsgs'),inp=document.getElementById('vcIn'),
      btn=document.getElementById('vcSend'),chips=document.getElementById('vcChips');

  function esc(s){var d=document.createElement('div');d.textContent=s;return d.innerHTML}

  function md(t){
    return t
      .replace(/```(\w*)\n([\s\S]*?)```/g,function(_,l,c){return '<pre><code>'+esc(c)+'</code></pre>'})
      .replace(/`([^`]+)`/g,'<code>$1</code>')
      .replace(/\*\*([^*]+)\*\*/g,'<strong>$1</strong>')
      .replace(/\*([^*]+)\*/g,'<em>$1</em>')
      .replace(/^#### (.+)$/gm,'<h4>$1</h4>')
      .replace(/^### (.+)$/gm,'<h4>$1</h4>')
      .replace(/^## (.+)$/gm,'<h3>$1</h3>')
      .replace(/^\d+\. (.+)$/gm,'<li>$1</li>')
      .replace(/^- (.+)$/gm,'<li>$1</li>')
      .replace(/\n\n/g,'<br><br>')
      .replace(/\n/g,'<br>');
  }

  function addMsg(text,isUser){
    var m=document.createElement('div');m.className='vc-msg '+(isUser?'user':'bot');
    var b=document.createElement('div');b.className='vc-bbl';
    b.innerHTML=isUser?esc(text):md(text);
    m.appendChild(b);msgs.appendChild(m);msgs.scrollTop=msgs.scrollHeight;
  }

  function showTyping(){
    var m=document.createElement('div');m.className='vc-msg bot';m.id='vcTyping';
    m.innerHTML='<div class="vc-bbl"><div class="vc-typing"><span></span><span></span><span></span></div></div>';
    msgs.appendChild(m);msgs.scrollTop=msgs.scrollHeight;
  }

  function hideTyping(){var e=document.getElementById('vcTyping');if(e)e.remove()}

  window.vcSendMsg=function(){
    var q=inp.value.trim();if(!q)return;
    inp.value='';chips.style.display='none';
    addMsg(q,true);showTyping();btn.disabled=true;
    google.colab.kernel.invokeFunction('notebook_chat',[q],{})
      .then(function(r){
        hideTyping();
        var a=r.data['application/json'];
        addMsg(typeof a==='string'?a:JSON.stringify(a),false);
      })
      .catch(function(){
        hideTyping();
        addMsg('Sorry, I encountered an error. Please check your internet connection and try again.',false);
      })
      .finally(function(){btn.disabled=false;inp.focus()});
  };

  window.vcAsk=function(q){inp.value=q;vcSendMsg()};
  window.vcClear=function(){
    msgs.innerHTML='<div class="vc-msg bot"><div class="vc-bbl">&#128075; Chat cleared. Ask me anything!</div></div>';
    chips.style.display='flex';
  };

  inp.addEventListener('keypress',function(e){if(e.key==='Enter')vcSendMsg()});
  inp.focus();
})();
</script>'''))